In [1]:
from darts import TimeSeries
!pip install pandas
import pandas as pd
import tqdm as notebook_tqdm

In [2]:
df = pd.read_csv('C:/Users/ashwini.kumar/Downloads/AirPassengers.csv')

Series = TimeSeries.from_dataframe(df, 'Month', '#Passengers')

In [10]:
# Create a TimeSeries, specifying the time and value columns
series = TimeSeries.from_dataframe(df, 'Month', '#Passengers')

# Set aside the last 36 months as a validation series
train_data, val_data = series[:-36], series[-36:]

In [ ]:
## First We will try the DART Model and see how it behaves

In [6]:
from darts.models import NBEATSModel

In [7]:
# creating a model object
model = NBEATSModel(input_chunk_length=24 , output_chunk_length=12, n_epochs = 2 , random_state = 15)

In [8]:
# fitting the model
model = model.fit(train_data, verbose = True)


[2022-03-17 20:19:23,183] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 73 samples.
[2022-03-17 20:19:23,183] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 73 samples.
[2022-03-17 20:19:23,258] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-03-17 20:19:23,258] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-03-17 20:19:23,259] WARNING | darts.models.forecasting.torch_forecasting_model | DeprecationWarning: kwarg `verbose` is deprecated and will be removed in a future Darts version. Instead, control verbosity with PyTorch Lightning Trainer parameters `enable_progress_bar`, `progress_bar_refresh_rate` and `enable_model_summary` in the `pl_trainer_kwargs` dict at model creation.
[2022-03-17 20:19:23,259] WARNING | darts.models.forecasting.torch_forecasting_model | Deprecati

Epoch 1: 100%|████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  5.11it/s, loss=1.93e+04]


In [9]:
# imports
from darts.metrics import mape

pred_air = model.predict(n = 36, series = train_data)


print("Mape = {:.2f}%".format(mape(val_data , pred_air)))


Predicting: 3it [00:00, ?it/s]
Mape = 17.55%


In [10]:
print("Mape = {:.2f}%".format(mape(val_data , pred_air)))

Mape = 17.55%


In [12]:
!pip install wandb -qqq
import wandb

In [11]:
wandb.login()

NameError: name 'wandb' is not defined

In [13]:
wandb.init()  
wandb.log({'accuracy_score': mape(val_data , pred_air)})

In [14]:
import wandb

sweep_config = {
  "name" : "my-sweep",
  "method" : "random",
  "parameters" : {
    "n_epochs" : {
      "values" : [10, 20, 50]
    },
      "input_chunk_length" : {"values" : [18,24,30,36]},
      "output_chunk_length" : {"values" : [9,12,15,18]}
  }
}

sweep_id = wandb.sweep(sweep_config)
import pprint
pprint.pprint(sweep_config)

Create sweep with ID: xmja2gsn
Sweep URL: https://wandb.ai/fibonacciash/uncategorized/sweeps/xmja2gsn
{'method': 'random',
 'name': 'my-sweep',
 'parameters': {'input_chunk_length': {'values': [18, 24, 30, 36]},
                'n_epochs': {'values': [10, 20, 50]},
                'output_chunk_length': {'values': [9, 12, 15, 18]}}}


In [15]:
def train():
    with wandb.init() as run:
        config = wandb.config

       
 # your model training code here
        for epoch in [80,120]:
            model = NBEATSModel(input_chunk_length=config.input_chunk_length , output_chunk_length=config.output_chunk_length,
                                n_epochs = epoch,
                            random_state = 15)
            
            # fitting the model
            model_req = model.fit(train_data, verbose = True)
            
            pred_air = model_req.predict(n = 58, series = train_data)


            loss = mape(val_data , pred_air)
            wandb.log({"loss": loss, "epoch": epoch})           

count = 5 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: l6tjito8 with config:
wandb: 	input_chunk_length: 18
wandb: 	n_epochs: 50
wandb: 	output_chunk_length: 9


[2022-03-17 20:19:43,672] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 82 samples.
[2022-03-17 20:19:43,672] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 82 samples.
[2022-03-17 20:19:43,749] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-03-17 20:19:43,749] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-03-17 20:19:43,751] WARNING | darts.models.forecasting.torch_forecasting_model | DeprecationWarning: kwarg `verbose` is deprecated and will be removed in a future Darts version. Instead, control verbosity with PyTorch Lightning Trainer parameters `enable_progress_bar`, `progress_bar_refresh_rate` and `enable_model_summary` in the `pl_trainer_kwargs` dict at model creation.
[2022-03-17 20:19:43,751] WARNING | darts.models.forecasting.torch_forecasting_model | Deprecati

Epoch 9: 100%|████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  5.64it/s, loss=1.28e+03]
Predicting: 3it [00:00, ?it/s]

[2022-03-17 20:19:49,397] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 82 samples.
[2022-03-17 20:19:49,397] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 82 samples.


[2022-03-17 20:19:49,490] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-03-17 20:19:49,490] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-03-17 20:19:49,492] WARNING | darts.models.forecasting.torch_forecasting_model | DeprecationWarning: kwarg `verbose` is deprecated and will be removed in a future Darts version. Instead, control verbosity with PyTorch Lightning Trainer parameters `enable_progress_bar`, `progress_bar_refresh_rate` and `enable_model_summary` in the `pl_trainer_kwargs` dict at model creation.
[2022-03-17 20:19:49,492] WARNING | darts.models.forecasting.torch_forecasting_model | DeprecationWarning: kwarg `verbose` is deprecated and will be removed in a future Darts version. Instead, control verbosity with PyTorch Lightning Trainer parameters `enable_progress_bar`, `progress_bar_refresh_rate` and `enable_model_summary` in 

...

Epoch 29: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  4.99it/s, loss=205]
Predicting: 3it [00:00, ?it/s]

[2022-03-17 20:25:49,999] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 73 samples.
[2022-03-17 20:25:49,999] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 73 samples.


[2022-03-17 20:25:50,114] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-03-17 20:25:50,114] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-03-17 20:25:50,115] WARNING | darts.models.forecasting.torch_forecasting_model | DeprecationWarning: kwarg `verbose` is deprecated and will be removed in a future Darts version. Instead, control verbosity with PyTorch Lightning Trainer parameters `enable_progress_bar`, `progress_bar_refresh_rate` and `enable_model_summary` in the `pl_trainer_kwargs` dict at model creation.
[2022-03-17 20:25:50,115] WARNING | darts.models.forecasting.torch_forecasting_model | DeprecationWarning: kwarg `verbose` is deprecated and will be removed in a future Darts version. Instead, control verbosity with PyTorch Lightning Trainer parameters `enable_progress_bar`, `progress_bar_refresh_rate` and `enable_model_summary` in 

Epoch 39: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  4.00it/s, loss=179]
Predicting: 3it [00:00, ?it/s]



epoch,▁▃▆█
loss,█▄▄▁
epoch,40
loss,13.174
